# pega-links

Esse script coleta os links para os pronunciamentos de um determinado parlamentar que estão registrados no [banco de dados de discursos e notas taquigráficas](https://www2.camara.leg.br/atividade-legislativa/discursos-e-notas-taquigraficas) da Câmara dos Deputados.

#### Importação de pacotes

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import pprint as pp
import time, random


#### Funções para executar a busca

In [2]:
def build_url(txOrador = '', 
             txPartido = '', 
             txUf = '',
             CurrentPage = '', 
             dtInicio = '', 
             dtFim = '',
             txTexto = '',
             txSumario = '',
             basePesq  = 'plenario',
             CampoOrdenacao = 'dtSessao',
             PageSize = '50',
             TipoOrdenacao = 'DESC'):
    
    '''
    Essa função cria a url de busca no sistema de arquivo da Câmara.
    Ao menos um dos campos cujo valor default é '' precisa ser preenchido.
    '''
    
    search_values = [ True for item in [txOrador, txPartido, txUf, CurrentPage, dtInicio, dtFim, txTexto, txSumario] if item != '' ]
    
    if not any(search_values):
        raise Exception("Para a URL retornar uma requisição válida, é necessário passar ao menos um parâmetro de busca.")
    
    # url de busca
    base_url = "https://www.camara.leg.br/internet/sitaqweb/resultadoPesquisaDiscursos.asp?"
    
    # Essa síntaxe define uma string longa sem quebras de linhas ou espaços
    params = (
    "txOrador={txOrador}"
    "&txPartido={txPartido}"
    "&txUF={txUF}"
    "&CurrentPage={CurrentPage}"
    "&dtInicio={dtInicio}"
    "&dtFim={dtFim}"
    "&txTexto={txTexto}"
    "&txSumario={txSumario}"
    "&basePesq={basePesq}"
    "&CampoOrdenacao={CampoOrdenacao}"
    "&PageSize={PageSize}")
    
    # Substitui os valores na string param
    params = params.format(
        txOrador = txOrador, 
        txPartido = txPartido, 
        txUF = txUf,
        CurrentPage = CurrentPage,
        dtInicio = dtInicio, 
        dtFim = dtFim, 
        txTexto = txTexto,
        txSumario = txSumario, 
        basePesq = basePesq, 
        CampoOrdenacao = CampoOrdenacao, 
        PageSize = PageSize, 
        TipoOrdenacao = TipoOrdenacao
    )
    
    # Compõe url final
    url = base_url + params
    return url

In [3]:
def make_request(url):
    
    '''
    Faz uma requisição para a url desejada
    e retorna um valor textual
    '''
    
    r = requests.get(url)
    r.encoding = 'UTF-8' # Muda encoding para evitar erros de caractere
    
    return r.text

#### Raspagem

In [4]:
def make_soup(data):
    '''
    Recebe uma string de texto e retorna um objeto do BeautifulSoup.
    '''
    soup = BeautifulSoup(data, 'html.parser')
    return soup

In [5]:
def scrape_table(soup, basePesq):
    
    '''
    Pega os dados da tabela na paginação atual
    e salva para uma lista passado como parâmetro.
    
    Parâmetros:
    
    soup -> Documento parseado pelo BeautifulSoup
    database -> String que determina em que base
    de dados a busca deve ser executada: 'plenario'
    ou 'comissao'.
    '''
    
    if basePesq == 'plenario':
        data = scrape_plen(soup)
        
    elif basePesq == 'comissao':
        data = scrape_com(soup)
        
    return data

In [6]:
def scrape_plen(soup):
    
    '''
    Função que raspa dados das manifestações
    do parlamentar em plenário.
    
    Parâmetros:
    
    soup -> Documento parseado pelo BeautifulSoup
    '''
    
    # Encontra a tabela
    table = soup.find('table')

    # Encontra os cabeçalhos
    headers = table.find_all('th')

    # Cria uma lista para cada cabeçalho em um dicionário
    data = { header.text : [] for header in headers }

    # Acessa apenas o corpo da tabela
    table_body = table.find('tbody')

    # Acessa cada linha do corpo da tabela
    rows = table_body.find_all('tr')
        
    # Filtra linhas estranhas que não carregam dados
    rows = [ row for row in rows if not row.has_attr("name") ]

    for row in rows:         

        # Encontra todas as células da tabela
        cells = row.find_all('td')

        # Sempre teremos oito células.
        # Podemos salvá-las na ordem da lista em seus respectivos campos.
        data["Data"].append(cells[0].text)
        data["Sessão"].append(cells[1].text)
        data["Fase"].append(cells[2].text)

        # Não há links para uma transcrição em html para dados mais antigos.
        # Assim, aso o script encontre um TypeError (por tentar acessar o atributo 
        # 'href de um elemento None-type), a saída é preencher com o texto.
        try:
            data["Discurso"].append(cells[3].find('a')['href'])
        except TypeError:
            data["Discurso"].append(cells[3].text)

        data["Sumário"].append(cells[4].find('a')['title'])
        data["Orador"].append(cells[5].text)
        data["Hora"].append(cells[6].text)

        # Mesmo problema do campo 'Discurso', com a mesma solução
        try:
            # Aqui, estamos acessando um atributo que tem os parâmetros
            # para uma função JavaScript que monta a URL onde podemos acessar o PDF
            data["Publicação"].append(cells[7].find('a')['onclick'])
        except (TypeError, KeyError): 
            data["Publicação"].append(cells[7].text)

    # Tira os brancos desnecessários de todas as listas
    # e faz alterações em sessões particulares
    for key, value in data.items():
        data[key] = [ item.strip() for item in value ]

        if key == 'Discurso':
            new_data = [ item.replace("\r\n\t\t\t\t\t\t\t", "") for item in value ]
            new_data = [ "https://www.camara.leg.br/internet/sitaqweb/" + item if item != '\xa0' else '-' for item in new_data ]
            data[key] = new_data

        if key == 'Sumário':
            new_data = [ item.replace("Sumário:\xa0", "") for item in value ]
            data[key] = new_data
            
        if key == 'Publicação':
            new_data = [ item.replace('MostraImagem','') if 'MostraImagem' in item else '' for item in value ]
            new_data = [ item.replace('(', '').replace(')', '') for item in new_data ]
            new_data = [ item.replace("'", "") for item in new_data]
            new_data = [ item.split(',') for item in new_data ] # Se não há o que splitar, retorna uma lista vazia
            new_data = [ build_pdf_link(item) for item in new_data]
            data[key] = new_data

    return data

In [7]:
def scrape_com(soup):

    '''
    Função que raspa dados das manifestações
    do parlamentar nas comissões.
    '''
    
    # Encontra a tabela
    table = soup.find('table')

    # Encontra os cabeçalhos
    headers = table.find_all('th')

    # Cria uma lista para cada cabeçalho em um dicionário
    data = { header.text : [] for header in headers }

    # Acessa apenas o corpo da tabela
    table_body = table.find('tbody')

    # Acessa cada linha do corpo da tabela
    rows = table_body.find_all('tr')
        
    # Filtra linhas estranhas que não carregam dados
    rows = [ row for row in rows if not row.has_attr("name") ]

    for row in rows:         

        # Encontra todas as células da tabela
        cells = row.find_all('td')

        # Sempre teremos oito células.
        # Podemos salvá-las na ordem da lista em seus respectivos campos.
        data["Data"].append(cells[0].text)
        data["Reunião"].append(cells[1].text)
        data["Tipo"].append(cells[2].text)
        data["Texto"].append(cells[3].find('a')['href'])
        data["Comissão"].append(cells[4].text)
        data["Hora"].append(cells[5].text)

    # Tira os brancos desnecessários de todas as listas
    # e faz alterações em sessões particulares
    for key, value in data.items():
        data[key] = [ item.strip() for item in value ]
        
        if key == 'Texto':
            new_data = [ item.replace("\r\n\t\t\t\t\t", "") for item in value ]
            new_data = [ "https://www.camara.leg.br/internet/sitaqweb/" + item if item != '\xa0' else '-' for item in new_data ]
            data[key] = new_data

    return data

In [8]:
def build_pdf_link(array):
    
    '''
    Com base no array de argumentos que retiramos de um parâmetro
    do campo 'Publicação' da tabela, podemos chamar uma função
    que reconstrói o link para acessar o PDF do discurso.
    
    '''
    
    if len(array) != 4:
        return 'no_link'
    
    # Precismamos transformar o primeiro elemento no
    # valor correto para gerar a url do PDF.
    if "DCD" in array[0]:
        array[0] = "D"
    elif "DCNR" in array[0]:
        array[0] = "R"
    elif "DCN" in array[0]:
        array[0] = "J"
    elif "DANC" in array[0]:
        array[0] = "R"
    elif "ANA" in array[0]:
        array[0] = "A"
        
    # Uma vez alterado o primeiro elemento, basta formatar
    # a seguinte url com base no array do parâmetros
    url = ("http://imagem.camara.gov.br/dc_20.asp"
           "?selCodColecaoCsv={}"
           "&txPagina={}"
           "&Datain={}"
           "&txSuplemento={}")
    
    # O asterisco permite passar uma lista de argumentos 
    # de tamanho desconhecido para o método .format (*args)
    url = url.format(*array)
    
    return url

#### Rodar raspador

In [9]:
def run_scraper(df_final,**kwargs):
    
    '''
    Função que encapsula as definidas anteriormente,
    converte o output para dataframe e salva como csv.
    '''

    url = build_url(CurrentPage = kwargs["CurrentPage"],
                    dtInicio = kwargs["dtInicio"],
                    dtFim    = kwargs["dtFim"],
                    PageSize = kwargs["PageSize"], 
                    basePesq = kwargs["basePesq"])
    
    #print(url)

    doc = make_request(url)
    soup = make_soup(doc)
    data = scrape_table(soup = soup, basePesq = kwargs["basePesq"])
    df = pd.DataFrame(data)
    print("\t", len(df))
    
    if len(df) > 0:
        fpath = "../data/tables/{dtInicio}-{basePesq}-metadata.csv"
        fpath = fpath.format(
            dtInicio = kwargs["dtInicio"].replace("/","-"),
            basePesq = kwargs["basePesq"],
        )
        df_final = pd.concat([df_final,df])
        df_final.to_csv(fpath, index = False)
        prox_pag = True
        
    else:
        prox_pag = False
    
    return df_final, prox_pag 

In [ ]:
#https://www.camara.leg.br/internet/SitaqWeb/ResultadoPesquisaDiscursos.asp?txOrador=&txPartido=&txUF=&dtInicio=01%2F01%2F2010&dtFim=30%2F12%2F2010&txSessao=&listaTipoFala=&listaFaseSessao=&listaTipoInterv=&txAparteante=&txTexto=&txSumario=&txIndexacao=&BasePesq=plenario&CampoOrdenacao=dtSessao&PageSize=20&TipoOrdenacao=ASC&btnPesqAvan=Pesquisar
https://www.camara.leg.br/internet/sitaqweb/resultadoPesquisaDiscursos.asp?txOrador=&txPartido=&txUF=&dtInicio=01/01/2010&dtFim=30/12/2010&txTexto=&txSumario=&basePesq=plenario&CampoOrdenacao=dtSessao&PageSize=1000
https://www.camara.leg.br/internet/sitaqweb/resultadoPesquisaDiscursos.asp?txIndexacao=&CurrentPage=2&BasePesq=plenario&txOrador=&txPartido=&dtInicio=01/01/2010&dtFim=30/12/2010&txUF=&txSessao=&listaTipoSessao=&listaTipoInterv=&inFalaPres=&listaTipoFala=&listaFaseSessao=&txAparteante=&listaEtapa=&CampoOrdenacao=dtSessao&TipoOrdenacao=&PageSize=1000&txTexto=&txSumario=    
https://www.camara.leg.br/internet/sitaqweb/resultadoPesquisaDiscursos.asp?txOrador=&txPartido=&txUF=&CurrentPage=2&dtInicio=01/01/2010&dtFim=30/12/2010&txTexto=&txSumario=&basePesq=plenario&CampoOrdenacao=dtSessao&PageSize=1000
    
    


In [11]:
# A célula abaixo executa o código e salva outputs no formato csv
'''
datas = [#("01/01/2010","30/12/2010"),
         ("01/01/2011","30/12/2011"),
         ("01/01/2012","30/12/2012"),
         ("01/01/2013","30/12/2013"),
         ("01/01/2014","30/12/2014"),
         ("01/01/2015","30/12/2015"),
         ("01/01/2016","30/12/2016"),
         ("01/01/2017","30/12/2017"),
         ("01/01/2018","30/12/2018"),
         ("01/01/2019","30/12/2019"),
        ]

'''
datas = [#("01/01/2001","30/12/2001"),
         ("01/01/2003","30/12/2003"),
         ("01/01/2005","30/12/2005"),
         ("01/01/2007","30/12/2007"),
         ("01/01/2019","30/12/2019")]

prox_pag = True

kwargs = {
    "dtInicio":"",
    "dtFim": "",
    "PageSize":"1000",
    "basePesq":"plenario",
    "CurrentPage": ""
}

for data in datas:
    df_final = pd.DataFrame()
    kwargs["dtInicio"] = data[0]
    kwargs["dtFim"] = data[1]
    print(kwargs)
    pag = 1
    prox_pag = True
    
    while(prox_pag):
        kwargs["CurrentPage"] = str(pag)
        print("\tDF_FINAL: ", len(df_final), prox_pag)

        time.sleep(random.randint(1,6))

        df_final, prox_pag = run_scraper(df_final, **kwargs)        
        pag += 1
        

{'dtInicio': '01/01/2003', 'dtFim': '30/12/2003', 'PageSize': '1000', 'basePesq': 'plenario', 'CurrentPage': ''}
	DF_FINAL:  0 True
	 1000
	DF_FINAL:  1000 True
	 1000
	DF_FINAL:  2000 True
	 1000
	DF_FINAL:  3000 True
	 1000
	DF_FINAL:  4000 True
	 1000
	DF_FINAL:  5000 True
	 1000
	DF_FINAL:  6000 True
	 1000
	DF_FINAL:  7000 True
	 1000
	DF_FINAL:  8000 True
	 1000
	DF_FINAL:  9000 True
	 1000
	DF_FINAL:  10000 True
	 1000
	DF_FINAL:  11000 True
	 1000
	DF_FINAL:  12000 True
	 1000
	DF_FINAL:  13000 True
	 1000
	DF_FINAL:  14000 True
	 1000
	DF_FINAL:  15000 True
	 1000
	DF_FINAL:  16000 True
	 1000
	DF_FINAL:  17000 True
	 1000
	DF_FINAL:  18000 True
	 1000
	DF_FINAL:  19000 True
	 1000
	DF_FINAL:  20000 True
	 1000
	DF_FINAL:  21000 True
	 1000
	DF_FINAL:  22000 True
	 1000
	DF_FINAL:  23000 True
	 902
	DF_FINAL:  23902 True
	 0
{'dtInicio': '01/01/2005', 'dtFim': '30/12/2005', 'PageSize': '1000', 'basePesq': 'plenario', 'CurrentPage': '25'}
	DF_FINAL:  0 True
	 1000
	DF_FINAL:  1

In [ ]:
# Altera a base de pesquisa e roda novamente
kwargs["basePesq"] = "comissao" #https://www.camara.leg.br/internet/sitaqweb/resultadoPesquisaDiscursos.asp?txOrador=&txPartido=&txUF=&CurrentPage=&dtInicio=01/01/2011&dtFim=30/12/2011&txTexto=&txSumario=&basePesq=plenario&CampoOrdenacao=dtSessao&PageSize=10
run_scraper(**kwargs)